## modelo SVR

In [6]:

import os
import numpy as np
import pandas as pd
import joblib
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler

def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i + look_back])
        y.append(series[i + look_back + passo - 1])
    return np.array(X), np.array(y)

def treinar_svr_para_componentes_d():
    arquivos_componentes = {
        'd1': 'D1_component.csv',
        'd2': 'D2_component.csv',
        'd3': 'D3_component.csv'
    }

    passos = [1, 5, 7, 30]
    os.makedirs("modelosSVR", exist_ok=True)
    os.makedirs("scalersSVR", exist_ok=True)

    for comp, arquivo in arquivos_componentes.items():
        df = pd.read_csv(arquivo)
        serie = df[comp.upper()].values.reshape(-1, 1)
        serie = pd.Series(serie.flatten()).interpolate().ffill().bfill().values.reshape(-1, 1)

        for passo in passos:
            look_back = 5 if passo <= 5 else 10

            # Normalização
            scaler = MinMaxScaler()
            serie_scaled = scaler.fit_transform(serie)
            joblib.dump(scaler, f"scalersSVR/scaler_{comp}_t{passo}.joblib")

            # Criar dataset
            X, y = criar_dataset_multi_step(serie_scaled, look_back, passo)
            split = int(len(X) * 0.8)
            X_train, y_train = X[:split], y[:split]

            # Corrigir dimensão
            X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]))


            # Treinar SVR
            model = SVR(C=10, epsilon=0.01, kernel='rbf')
            model.fit(X_train, y_train.ravel())

            # Salvar modelo
            joblib.dump(model, f"modelosSVR/svr_{comp}_t{passo}.joblib")
            print(f"✅ Modelo salvo: svr_{comp}_t{passo}.joblib")

# Executar o treinamento
treinar_svr_para_componentes_d()


✅ Modelo salvo: svr_d1_t1.joblib
✅ Modelo salvo: svr_d1_t5.joblib
✅ Modelo salvo: svr_d1_t7.joblib
✅ Modelo salvo: svr_d1_t30.joblib
✅ Modelo salvo: svr_d2_t1.joblib
✅ Modelo salvo: svr_d2_t5.joblib
✅ Modelo salvo: svr_d2_t7.joblib
✅ Modelo salvo: svr_d2_t30.joblib
✅ Modelo salvo: svr_d3_t1.joblib
✅ Modelo salvo: svr_d3_t5.joblib
✅ Modelo salvo: svr_d3_t7.joblib
✅ Modelo salvo: svr_d3_t30.joblib
